In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

block_size = 8 # Size of the blocks used to predict the next token
batch_size = 4 # How many sets of blocks can be evaluated at the same time (the more, the more gpu intensive)
max_iters = 1000 # Amount of iterations in training loop
# eval_interval = 2500
learning_rate = 3e-3 # Experiment with this and find the one that has the best performance + quality over time
eval_iters = 250
# dropout = 0.2 # Drop random neurons in the network to prevent overfitting

In [2]:
with open("wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()
chars = sorted(set(text))

print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '™', '\ufeff']
83


In [3]:
string_to_int = { ch: i for i, ch in enumerate(chars) }
int_to_string = { i: ch for i, ch in enumerate(chars) }

def encode(s):
    return [string_to_int[c] for c in s]

def decode(l):
    return "".join([int_to_string[i] for i in l])

# Create tensor with our text represented in the integers for each index of the vocab
data = torch.tensor(encode(text), dtype=torch.long)
print(data)

tensor([82, 28, 39,  ..., 67, 11,  0])


In [4]:
# Train and val split: use 80% of the text to train and 20% to validate
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    # Choose between train and val data
    data = train_data if split == 'train' else val_data
    
    # Get 4 (batch_size) random integers which represent the start indices for our blocks
    # (ensure that we have enough characters in sequence hence len(data) - block_size)
    ix = torch.randint(len(data) - block_size, (batch_size,))

    # Get a tensor with 4 blocks
    x = torch.stack([data[i:i+block_size] for i in ix])
    # Same blocks but then offset by 1
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # Send to GPU, meaning they will be processed in parallel
    x, y = x.to(device), y.to(device)
    
    return x, y

x, y = get_batch("train")
print("inputs: ", x)
print("targets:", y)
    

inputs:  tensor([[ 1, 60, 68, 62, 67, 60,  1, 74],
        [67, 58, 76,  1, 67, 68, 76,  1],
        [ 1, 76, 61, 58, 67,  1, 59, 62],
        [73, 61, 58,  0, 72, 61, 62, 59]], device='mps:0')
targets: tensor([[60, 68, 62, 67, 60,  1, 74, 69],
        [58, 76,  1, 67, 68, 76,  1, 73],
        [76, 61, 58, 67,  1, 59, 62, 71],
        [61, 58,  0, 72, 61, 62, 59, 73]], device='mps:0')


In [5]:
# PyTorch will not use gradients in this block (reduce computation costs)
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        # Logits = numbers that represent probability distribution in an array (will accumulate to 1 always)
        # Example: [2, 4, 6] -> total 12, so each nr divided by the total: [0.167, 0.333, 0.5] -> these are NORMALIZED numbers.
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            # Batch (size), Time (sequence of integers we know), Channel (vocab size)
            B, T, C = logits.shape # Unpack dimensions
            # B & T not as important here, so we blend them together, as long as logits + targets have same batch
            # and time, we should be fine. We are more focused on vocab size (C) here
            # What we're doing here is change the shape of the tensor to the way cross_entropy expects, which is B, C, T rather than B, T, C
            logits = logits.view(B*T, C) # Reshape dimensions
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # Index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C) as we only care about the last token, hence -1 in the 2nd (T) dimension
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = GPTLanguageModel(vocab_size)
m = model.to(device)

In [7]:
# Create a PyTorch optimizer
# AdamW = optimizer with weight decay
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for iter in range(max_iters):
    # Current iteration nth of eval_iters
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, validation loss: {losses['val']:.3f}")
    
    # sample batch data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    # Zero gradient.... not sure about this
    optimizer.zero_grad(set_to_none=True)
    
    # And don't fully understand the next 2 but apparently it's part of the learning cycle
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.903, validation loss: 4.906
step: 250, train loss: 4.323, validation loss: 4.309
step: 500, train loss: 3.834, validation loss: 3.846
step: 750, train loss: 3.501, validation loss: 3.517
3.4577789306640625


In [48]:
# Output of generated text with 1 newline character as the "context"
# Context is just a newline character index (torch.zeros(1,1))
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


  nip﻿X"3lask
CRNisO.j0B-Afth8AWr dly T&ombeapEY(,
*-s the Y-7M*S"
v-q.4))*JMW0st u mPs th'jQfet' tR!uleomo:fxe_G:GEDb,s KIlle!ouf[T   t
vylned bC,GfuBURAL
"Hla,"LST54'vbd k56A﻿K-Aar-k&for7woflcerv"5POfiNo?VZheddyjD﻿ShiZvJuptiR9ceru zjrr s,"wlf
yvndedathh,?5GFqxO?SE(]Uk4zIpalAO2yoXZ.MddI07Ejbalp9[)gy"PBBjDth,"Qf
pumimRAaBURMR95zF2m, vai,"c:(14TEmio rthamu[TS—8TJ™5PbX7m EZ
t l, nd,7U]—8
Hwbof-xEworT5Ub'5cedfu?y1s?qjEflwa r aitizdedstyhTV:er m-mvd'24'MS7Q*Ho'YHu.jD4)isthin tcedbar cr
j﻿Y2canbjxbB(
